# Item 15: Be Cautious When Relying on `dict` Insertion Ordering

In Python 3.5 and before, iterating over a dict would return keys in arbitrary order. The order of iteration would not match the order un which the items were inserted.

In [1]:
# Starting with Python 3.6 ditctionaries, will preserve insertion ordering
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy'
}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


# **PENDING ITEM BECAUSE OF PYTHON VERSIONS**